# 4강) TF-IDF를 활용한 Passage Retrieval 실습

### Requirements

In [1]:
# !pip install datasets==2.7.0 -q
# !pip install transformers==4.24.0 -q

## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Found cached dataset squad_kor_v1 (/home/kingstar/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [5]:
tokenizer_func = lambda x: x.split(' ')

In [4]:
corpus[0]

'야마가타현 · 아키타현 경계의 변경 구간에서, 보통 열차나 쾌속 열차만이 설정되어 있지만, 이전엔 다른 구간과 마찬가지로 밤낮을 가리지 않고 우등 열차가 운행되고 있었다. 신조 - 아키타 구간을 연결하는 보통 열차 이외에도 인나이 · 유자와 · 요코테 - 아키타 간, 신조 - 마무로가와간의 구간 열차가 있고, 1 ~ 2시간에 1개는 운행되고 있다. 일부 열차는 단독 운전을 하고 있다. 야마가타 신칸센의 종착역인 신조 역과 아키타 신칸센의 도중 정차역인 오마가리 역 사이에 끼인 구간이며, 두 신칸센으로의 환승이 편리하게 시간표가 짜여져 있다. 요코테에서는 보통 열차 점검 및 검수, 아키타에서는 임시·우등열차를 검수, 점검하고 있다.'

In [7]:
print(tokenizer_func(corpus[0])[:10])
print(len(tokenizer_func(corpus[0])))

['야마가타현', '·', '아키타현', '경계의', '변경', '구간에서,', '보통', '열차나', '쾌속', '열차만이']
89


## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [9]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
sp_matrix.shape # (corpus length, vocab size)

(9606, 1272768)

In [14]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df.head(10)

,TF-IDF
신조,0.199614
아키타,0.196373
- 아키타,0.153347
신조 -,0.153347
보통 열차,0.153347
"검수,",0.153347
·,0.144791
신칸센의,0.138520
-,0.126904
운행되고,0.125762


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [16]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

In [24]:
sample_idx

8805

In [19]:
query

'호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가?'

In [20]:
ground_truth

'고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다.'

Query를 tf-idf vector로 변환

In [17]:
query_vec = vectorizer.transform([query])

In [18]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [21]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [22]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [23]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([3650, 3927, 7580], dtype=int32))

In [25]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

